In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_case as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.CLIPS


c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]
# ] + train.Metrics.STATISTICAL

In [4]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


random.seed(10)

model_phases = dr.CLIPS

label_type = dr.SelfReports.AROUSAL
threshold = "fixed"

x, y = train.Train_CASE.get_case_data(
    metrics, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False, standardize=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.66662065 0.67057428 0.68128258 0.67960735 0.6870231  0.67049145
 0.67910148 0.69209773 0.67946127 0.69016394 0.66807274 0.67802367
 0.67873342 0.67692619 0.6768031  0.6548516  0.68444016 0.6869617
 0.68149672 0.68170153 0.67059641 0.68035336 0.67915068 0.68237009
 0.68362024        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
LGB: {'max_depth': 5, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [34 14], Predictions: [0 1], [36 12]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [34 14], Predictions: [0 1], [30 18]
Training RF ...
Model RF, Actual: [0 1], [34 14], Predictions: [0 1], [31 17]
Training XGB ...
Model XGB, Actual: [0 1], [34 14], Predictions: [0 1], [31 17]

None


[('rmssd', 126), ('sdnn', 119), ('hf_rr', 110), ('lf_rr', 100), ('lf_hf_ratio', 93), ('bpm', 82), ('mean_SCL', 81), ('SCR_rate', 48)]


[('lf_rr', 0.20439156339187659), ('hf_rr', 0.201788004284757

One or more of the test scores are non-finite: [0.6883729  0.70581838 0.70918002 0.71631631 0.71467746 0.6954504
 0.707462   0.71058246 0.71053226 0.70540767 0.72662824 0.7235038
 0.72898037 0.72389956 0.72121786 0.70233837 0.72216007 0.72571589
 0.71552237 0.71613646 0.71625427 0.71782328 0.7202367  0.71586652
 0.71584465        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [36 12], Predictions: [0], [48]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [36 12], Predictions: [0 1], [44  4]
Training RF ...
Model RF, Actual: [0 1], [36 12], Predictions: [0 1], [41  7]
Training XGB ...
Model XGB, Actual: [0 1], [36 12], Predictions: [0 1], [42  6]

None


[('rmssd', 76), ('sdnn', 75), ('hf_rr', 74), ('lf_rr', 73), ('lf_hf_ratio', 54), ('bpm', 51), ('mean_SCL', 40), ('SCR_rate', 27)]


[('lf_rr', 0.22601015138124525), ('hf_rr', 0.21376637088972747), (

...

Grid search for XGB ...
SVM: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [37 10], Predictions: [0 1], [45  2]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [37 10], Predictions: [0 1], [38  9]
Training RF ...
Model RF, Actual: [0 1], [37 10], Predictions: [0 1], [39  8]
Training XGB ...
Model XGB, Actual: [0 1], [37 10], Predictions: [0 1], [40  7]

None


[('sdnn', 102), ('hf_rr', 92), ('bpm', 56), ('lf_hf_ratio', 51), ('rmssd', 49), ('lf_rr', 45), ('mean_SCL', 32), ('SCR_rate', 27)]


[('hf_rr', 0.2113573795003806), ('lf_rr', 0.2057008545053569), ('b

One or more of the test scores are non-finite: [0.72080598 0.73942801 0.75096565 0.75180431 0.75381548 0.73790789
 0.76176833 0.75788534 0.76191351 0.75580901 0.75746622 0.7611614
 0.75509759 0.75249058 0.74589849 0.7455086  0.75343505 0.75190614
 0.75109096 0.75278851 0.75442972 0.76543851 0.76172    0.76328202
 0.76586398        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [34 14], Predictions: [0 1], [44  4]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [34 14], Predictions: [0 1], [35 13]
Training RF ...
Model RF, Actual: [0 1], [34 14], Predictions: [0 1], [35 13]
Training XGB ...
Model XGB, Actual: [0 1], [34 14], Predictions: [0 1], [35 13]

None


[('sdnn', 91), ('hf_rr', 90), ('lf_hf_ratio', 69), ('mean_SCL', 62), ('lf_rr', 57), ('rmssd', 48), ('bpm', 44), ('SCR_rate', 22)]


[('lf_rr', 0.23500762352625026), ('hf_rr', 0.20700137689825115), 

One or more of the test scores are non-finite: [0.72499001 0.69995689 0.71341229 0.71962111 0.71708045 0.72677233
 0.706675   0.71350824 0.71016068 0.70877504 0.70381227 0.71067879
 0.71107914 0.71331613 0.70977851 0.71337849 0.72261451 0.72092301
 0.72007456 0.71055115 0.71719213 0.72709404 0.72300555 0.72731052
 0.7161821         nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.001, 'kernel': 'sigmoid'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [34 14], Predictions: [0 1], [46  2]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [34 14], Predictions: [0 1], [41  7]
Training RF ...
Model RF, Actual: [0 1], [34 14], Predictions: [0 1], [43  5]
Training XGB ...
Model XGB, Actual: [0 1], [34 14], Predictions: [0 1], [41  7]

None


[('sdnn', 96), ('lf_rr', 76), ('hf_rr', 75), ('bpm', 68), ('rmssd', 61), ('lf_hf_ratio', 58), ('mean_SCL', 42), ('SCR_rate', 26)]


[('lf_rr', 0.22686524999929505), ('hf_rr', 0.207104191436612

In [26]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from scipy.stats import mode
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

random.seed(42)

def train_predict_ensemble(ensemble_models, x_train, y_train, x_test, y_test, features, type="majority_vote", weights=None):
    features = list(features.values())[0]
    # y_preds = []
    # # print(features)
    # for model_name in ensemble_models:
    #     x_test_temp = x_test.loc[:, features[model_name]]
    #     y_pred = ensemble_models[model_name].predict(x_test_temp)
    #     y_preds.append(y_pred)
    # if type == "majority_vote":
    #     y_preds = mode(y_preds, axis=0)[0]
    #     y_preds = np.reshape(y_preds, (y_preds.shape[1], 1))
    # elif type == "weighted_avg":
    #     if weights is None:
    #         weights = [1 for _ in y_preds]
    #     for i in range(len(y_preds)):
    #         y_preds[i] = y_preds[i]*weights[i]
    #     y_preds = np.rint(np.mean(y_preds, axis=0))

    ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
    ensemble = VotingClassifier(estimators=ensemble_models, voting='soft', weights=weights)
    ensemble.fit(x_train.loc[:, features], y_train)
    y_preds = ensemble.predict(x_test.loc[:, features])

    return y_preds


test_size = 0.15
folds = 5
ensemble_models = [
    # "SVM", 
    "LGB",
    "RF",
    "XGB"
]
voting_type = "weighted_avg"
weights = [
    0.8,
    1.0,
    1.0
]

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds, random_seed=45)
y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models[name] for name in ensemble_models}
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

	Accuracy: 0.8205128205128205
	Precision: 0.75
	Recall: 0.45
	F1-score: 0.5625000000000001
	AUC score: 0.6991379310344827
----------------------------------------
